$\qquad$ $\qquad$$\qquad$  **BBM409 Machine Learning: Assignment 3** <br />
$\qquad$ $\qquad$$\qquad$ **Goal: Neural Network**<br />
$\qquad$ $\qquad$$\qquad$                   **MELTEM TOKGÖZ** <br />
$\qquad$ $\qquad$$\qquad$                     **21527381** <br />
$\qquad$ $\qquad$$\qquad$                   **b21527381@cs.hacettepe.edu.tr** <br />

# PART I: Theoretical problems

**Question1**: What are differences between logistic regression and linear regression?  <br />
**SOLUTION :**<br />
* In linear regression, the outcome (dependent variable) is continuous. It can have any one of an infinite number of possible values. In logistic regression, the outcome (dependent variable) has only a limited number of possible values.<br />
* Linear regression gives an equation which is of the form Y = mX + C, means equation with degree 1. However, logistic regression gives an equation which is of the form Y = eX + e-X <br />
* Linear regression aims at finding the best-fitting straight line which is also called a regression line. In the above figure, the red diagonal line is the best-fitting straight line and consists of the predicted score on Y for each possible value of X. The distance between the points to the regression line represent the errors.<br />
<img src ="log.png"><br />
* Changing the coefficient leads to change in both the direction and the steepness of the logistic function. It means positive slopes result in an S-shaped curve and negative slopes result in a Z-shaped curve.<br />
<img src ="logreg.png"><br />

**Question2:** What are differences between logistic regression and naive bayes methods? <br />
**SOLUTION:**<br />
* The learning mechanism is a bit different between the two models, where Naive Bayes is a generative model and Logistic regression is a discriminative model. What does this mean?<br />  Generative model: Naive Bayes models the joint distribution of the feature X and target Y, and then predicts the posterior probability given as P(y|x)<br /> Discriminative model: Logistic regression directly models the posterior probability of P(y|x) by learning the input to output mapping by minimising the error.<br />
* Naïve Bayes: When the training data size is small relative to the number of features, the information/data on prior probabilities help in improving the results<br /> Logistic regression: When the training data size is small relative to the number of features, including regularisation such as Lasso and Ridge regression can help reduce overfitting and result in a more generalised model.<br />
* Naïve Bayes assumes all the features to be conditionally independent. So, if some of the features are in fact dependent on each other (in case of a large feature space), the prediction might be poor.<br /> Logistic regression splits feature space linearly, and typically works reasonably well even when some of the variables are correlated.<br />
* Summaries: When naive bayes estimates a joint probability from the training data, logistic regression estimates the probability from the training data by minimizing error. The other difference is limitations. When naive bayes can even with less training data, the logistic regression, model estimates may over fit the data. In addition, when the naive bayes assumes all the features are conditionally independent, logistic regression splits feature space linearly.<br />

**Question3:** Which of the following statements are true? <br />
**SOLUTION:**<br />
* A two layer (one input layer, one output layer; no hidden layer) neural network can represent the XOR function.<br />**False**<br />We must compose multiple logical operations by using a hidden layer to represent the XOR function.<br />XOR not linearly separable.<br /><img src ="xor2.jpg"><br />
* Any logical function over binary-valued (0 or 1) inputs x1andx2 can be (approximately) represented using some neural network.<br />**True**<br />Since we can build the basic AND, OR, and NOT functions with a two layer network, we can (approximately) represent any logical function by composing these basic functions over multiple layers.<br />
* Suppose you have a multi-class classiffication problem with three classes,trained with a 3 layer network. Let a(3)1 = (h(x))1 be the activation of the first output unit and similarly a(3) 2 = (h(x))2 and a(3) 3 = (h(x))3. Then for any input x, it must be the case that that a(3) 1 + a(3) 2 + a(3) 3 = 1.<br />**False**<br />The outputs of a neural network are not probabilities, so their sum need not be 1.<br />
* The activation values of the hidden units in a neural network, with the sigmoid activation function applied at every layer, are always in the range (0, 1).<br />**True**<br />


**Question4:** How to decide the number of hidden layers and nodes in a hidden layer?  <br />
**SOLUTION:**<br />
* There is no hard-and-fast rule for this.<br />  The number of hidden nodes you should have is based on a complex relationship between<br /> Number of input and output nodes<br />Amount of training data available<br />Complexity of the function that is trying to be learned<br />The training algorithm<br /> 
* To minimize the error and have a trained network that generalizes well, you need to pick an optimal number of hidden layers, as well as nodes in each hidden layer. <br />Too few nodes will lead to high error for your system as the predictive factors might be too complex for a small number of nodes to capture <br />Too many nodes will overfit to your training data and not generalize well<br />
* The number of hidden nodes in each layer should be somewhere between the size of the input and output layer, potentially the mean.<br />The number of hidden nodes shouldn't need to exceed twice the number of input nodes, as you are probably grossly overfitting at this point.<br />

In [0]:
#********************************************************************************************************************************

# PART II: Classification of Flowers using Neural Network

In this part of the assignment, I will implement a single layer and multilayer neural network architecture to classify flowers into 5 classses.

Firstly, I used to import the libraries I used.

In [0]:
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import argparse
import os

With this function I give the hyper parameters I want.

In [0]:
#***********************PARAMETERS*********************************************
def argumentParser():
    ap = argparse.ArgumentParser(formatter_class=argparse.RawTextHelpFormatter)
    ap.add_argument("-td", "--traindata", default='train.mat')
    ap.add_argument("-ttd", "--testdata", default='test.mat')
    ap.add_argument("-i", "--inputsize", default='768')
    ap.add_argument("-o", "--outputsize", default='5')
    ap.add_argument("-b", "--batchsize", default='128')
    ap.add_argument("-e", "--epoch", default='100')
    ap.add_argument("-lr", "--learningrate", default='0.01')
    return vars(ap.parse_args())

I've done a function to read and throw the data into matrices.Here :<br />
**label : ** 0-1-2-3-4 so total 5 class X 3000 image <br />
**image vector size : **  768 x 3000 image <br />Because it is important to normalize the image values from 0 to 1 (0 için 255), I divided it to 255.<br />There are other techniques to do normalization here.I've specified them in the comment line and I've tried trying my code with other techniques. But I didn't use it because it gave almost the same results.

In [0]:
# **********************READ DATA FUNCTION and NORMALIZATION***********************************************
def readData(trainFile, testFile):
    train = io.loadmat(trainFile)
    test = io.loadmat(testFile)

    ###TRAIN DATA###
    train_label = np.array(train['y'][0])
    train_vector = np.array(train['x']) / 255
    #normalization
    # train_vector = (train_vector -np.min(train_vector)) / (np.max(train_vector)-np.min(train_vector))
    # standartization
    # train_vector  = (train_vector - np.mean(train_vector)) / (10.0 * np.std(train_vector))

    ###TEST DATA###
    test_label = np.array(test['y'][0])
    test_vector = np.array(test['x']) / 255
    return train_label, train_vector, test_label, test_vector

This function to be correct class 1 other class 0 for each image.For example if correct class 2 , [0 0 1 0 0]

In [0]:
# ***************ONE HOT**************************************************
def oneHot(label, class_size):
    y = label.shape[0]
    label_matrix = np.zeros([y, class_size], dtype=int)
    for i in range(y):
        correct_class = label[i]
        label_matrix[i][correct_class] = 1
    return label_matrix

I wanted to try my code in different activation functions, so I defined different activation functions.<br />
**1-Sigmoid Function :** The formula is as follows:

<img src ="sigmoid.png"><br />

In [0]:
#**********************SIGMOID FUNCTION****************************************
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivative_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

**2-RELU Function :**  The formula is as follows: and result accuracy 16.06648199445983

<img src ="relu.png"><br />

In [0]:
# ************************ReLu FUNCTION****************************************
def relu(x):
    return np.maximum(x, 0)

def derivative_relu(x):
    x[x <= 0] = 0
    x[x > 0] = 1
    return x

**3- Tanh Function :**   The formula is as follows:

<img src ="tanh.png"><br />

In [0]:
#*************************TANH FUNCTION****************************************
def tanh(X):
    return np.tanh(X)
    
def derivative_tanh(x):
    return 1.0 - np.tanh(x) ** 2

**4- Softmax Function : **  The formula is as follows:

<img src ="soft.jpg"><br />

In [0]:
# ***************SOFTMAX FUNCTION**********************************************
def softmax(out_array):
    for i in range(out_array.shape[0]):
        out_array[i] = np.exp(out_array[i]) / np.sum(np.exp(out_array[i]), axis=0)
    return out_array

def derivative_softmax(preds):
    return preds * (1 - preds)

**LOSS FUNCTION**
The loss function basically calculates how different the prediction of the model is from the ground truth. 
As loss function, I used sum of negative log-likelihood of the correct labels in this assignment.

In [0]:
#*****************LOSS FUNCTION************************************************
def loss(train_label, a1, size):
    result = -np.sum(np.multiply(train_label, np.log(a1)) + np.multiply(1 - train_label, np.log(1 - a1))) / size
    return result

**The parts so far consisted of activation functions, loss function and data reading  for  I used both in single layer and in multilayer**<br />
Now I'm going to tell you how I defined a model on the single layer.

# 1 Single Layer Neural Network

In the code I wrote below, I will tell you the stages of the single layer one by one.<br />
* As a first step, I gave the initial values of weigt and bias.I gave them all 0 in bias, I used random function in weight.<br />
*  In the first place I give the necessary hyper parameters that call the function s_nn with args in main function. And I called s_nn function from main.<br />

**EPOCH COMMENT**<br />
While the model is being trained, all of the data is not included in the training at the same time. They take part in a number of parts. The first part is trained, the performance of the model is tested and the weights are updated with backpropagation. Then the model is re-trained with the new training set and the weights are updated again. This process is repeated in each training step to calculate the most suitable weight values for the model. Each of these training steps is called “epoch".<br />
* As the most suitable weight values were calculated step by step in my assignment, the performance was lower in the first epochs, and as the number of epoch increased, the performance increased. However, after a certain step, the learning status of my model will be considerably reduced.<br />
* For example 3000 epoch result: I print a accuracy value for every 100 epoch.<br />
<img src ="epoch.png"><br />
*  For example 500 epoch result: I print a accuracy value for every 100 epoch.<br />
<img src ="500epoch.png"><br />


**BATCH COMMENT**<br />
When designing the model, the value defined as mini-batch parameter means that the model will process the how many data at the same time.When we set the mini-batch value to 1 as the smallest value, we do the same job as  “schoastic gredient decent”. So in each iteration, we only process one data.In case the mini-bath value is equal to the number of all elements in the training set; all the data in the training set will enter training and the process will be the same as “batch gredient descent"<br />
* When I apply a mini-batch, there are fluctuations in the error value, as seen in the graph.<br />
** Below is the loss chart of my code which I tried with 3 different mini batch values.**<br />
* 1- mini batch value = 10<br /> It causes the model to learn the noise. So the zigzags in the chart are multiplying. Because different data is used each time during the learning process.<br /><img src ="10.png"><br />
* 2- mini batch value = 3000<br />The model learns less noise. Learning takes a little longer because it will process all the data at the same time.<br /><img src ="3000.png"><br />
* 3- mini batch value = 128 <br /> In selecting the mini-batch value, I have set a value that is neither too small nor too large between the most appropriate value 1 and the number of all data in the training set.<br /><img src ="128.png"><br />


Now, I implemented to single layer neural network algoritm.
First, I'm doing feed forward.

**FEED FORWARD COMMENT**<br />
I used follows formula this function.<br />
Formula : oi = wijxj + bi (here w=weight b=bias)<br />
Then I sent the output value from this process to the activation function.<br />

**ACTIVATION FUNCTION COMMENT**<br />
I have tried different activation functions in my code and got results.<br />
Accuracy values of the activation functions I use are as follows.<br />
I found that the best result is softmax funtion.That's why I'm using it in my code.<br />
**Softmax Function : ** result accuracy 31.57894736842105<br />
**Sigmoid Function : ** result accuracy 30.886426592797783<br />
**Tanh Function    : **  result accuracy 16.06648199445983<br />
**RELU Function    : **  result accuracy 16.06648199445983<br />

**BACKPROPAGATION COMMENT**<br />
In the backpropagation process, this update is called chain rule.<br />
Finding the difference by taking back the derivative and multiplying the difference value by the learning rate is calculated by subtracting the result from the weight values and calculating the new weight value.<br />
I find the new weight and bias values by taking back the derivative and multiplying them with the learning rate.First I take the derivative of the loss function and then the activation function I use.Then I am update weight and bias.<br />


**LEARNING RATE COMMENT**<br />
learning rate value 0.005 between 0.02<br />
Having a high learning rate means I'm very impressed by the data.<br />
I have tried different learning rate in my code and got results.<br />
* Learning rate : 0.005 Accuracy :  29.085872576177284<br />
* Learning rate : 0.02 Accuracy  :  29.916897506925206<br />
* Learning rate : 0.01 Accuracy  :  30.332409972299168<br />

**ACCURACY COMMENT**<br />
I'm applying feed forward to my test dataset with the latest weight and bias values.<br />
Then, I compare the test data set with the actual classes and find out how many correct classifications there are.<br />
With the following parameters(), I found the best accuracy 30.055401662049864

**NOTE!!!**<br />
My Teacher ,I've registered program as a model, but I'm not sure if I'm doing it right, so I'm writing the final parameters I've used against just in case I've already given them by default.<br />
batchsize =128<br />
epoch = 2000<br />
learningrate =0.01<br />
Activation function = Softmax<br />


**RESULT**<br />
I've divided it into 100 to optimize numbers in many places.<br />
The best accuracy is 30.05 in my single layer neural network.


In [0]:
# ******************************************************************************
#initialize weight and bias value
def initialize_parameters(input_s,output_s):
    initial_b = 0.0 #give zero
    initial_w = np.random.randn(input_s, output_s) #give random
    return initial_w, initial_b
#******************************************************************************
def feedforward(train_vector, initial_w, initial_b):
    # this formula : oi = wijxj + bi
    o1 = train_vector.dot(initial_w) + initial_b
    # send to activation function:
    a1 = softmax(o1 / 100)
    return a1
# ******************************************************************************
def backpropagation(train_label, train_vector, a1, initial_w, initial_b, learning_rate):
    # Backpropagation
    da1 = -(np.divide(train_label, a1) - np.divide(1 - train_label, 1 - a1)) / 100
    dh1 = da1 * derivative_softmax(a1)
    new_weight = train_vector.T.dot(dh1)
    new_bias = np.sum(dh1)

    # update weight and bias
    initial_w -= learning_rate * new_weight
    initial_b -= learning_rate * new_bias
    return initial_w, initial_b
# *****************************************************************************
def accuracy(test_vector,test_label,initial_w,initial_b):
    preds = softmax((test_vector.dot(initial_w) + initial_b) / 100)
    count = 0
    for i, j in zip(test_label, np.argmax(preds, axis=1)):
        if i == j:
            count += 1
    print("Accuracy : ")
    print(count / test_vector.shape[0] * 100)
# *****************************************************************************
def s_nn(train_vector, train_label, test_vector, test_label, batch_size, initial_w, initial_b, learning_rate,epoch_size):
    count = 0
    for epoch in range(epoch_size):
        g = []
        for i in range(0, len(train_vector), batch_size):
            size = train_vector[i:i + batch_size].shape[0]
            a1 = feedforward(train_vector[i:i + batch_size], initial_w, initial_b)
            # send to loss function:
            loss_val = loss(train_label[i:i + batch_size], a1, size)
            g.append(loss_val)
            intial_w, initial_b = backpropagation(train_label[i:i + batch_size], train_vector[i:i + batch_size]
                                                  , a1, initial_w, initial_b, learning_rate)
            count += 1

        if epoch % 100 == 0:
            preds = softmax((test_vector.dot(initial_w) + initial_b) / 100)
            count = 0
            for i, j in zip(test_label, np.argmax(preds, axis=1)):
                if i == j:
                    count += 1

            print(count / test_vector.shape[0] * 100)

    accuracy(test_vector, test_label, initial_w, initial_b)
     

    g.clear
    plt.plot(g)
    plt.show()
# ******************************************************************************  

**NOTES**<br />
My teacher, my code is running normally spyder or pycharm but ipython there is an error with args.If there is an error args here, you should know that you can run my code on other platforms(pycharm, spyder).

In [0]:
def main():
    args = argumentParser()
    train_label, train_vector, test_label, test_vector = readData(args['traindata'], args['testdata'])
    train_label = oneHot(train_label, int(args['outputsize']))
    initial_w , initial_b = initialize_parameters(int(args['inputsize']),int(args['outputsize']))         
    # other typhical value 0.001, 0.01, 0.03, 0.1, 0.3, 1 etc.
    #s_nn(train_vector, train_label, test_vector, test_label, int(args['batchsize']), initial_w, initial_b,float(args['learningrate']), int(args['epoch']))
    # save learned parameters
    if not os.path.isdir("../model"):
        os.mkdir("../model")
    np.savez("../model/model.npz", weights=initial_w, bias= initial_b )
    print("Parameters are saved..")
    # ******************************************************************************

if __name__ == '__main__':
    main()
    

# 2-Multi Layer Neural Network

Here, I did almost the same with the single layer.<br />
I could not make the number of layers dynamic, but my code works for 1 input layer and 1 hidden layer.<br />
I can dynamically change the number of neurons in the hidden layer. I tried the code with a different number of neurons.<br />
And I've observed that when there are more neurons, it gives better accuracy.<br />

With parameters<br />
epoch_size = 1000<br />
learning_rate = 0.01<br />
batch_size = 126<br />
output_size = 5<br />
input_size = 768<br />
hidden_size = 50 <br />

** result : ** 25.346260387811636<br />

With parameters<br />
epoch_size = 3000<br />
learning_rate = 0.005<br />
batch_size = 256<br />
output_size = 5<br />
input_size = 768<br />
hidden_size = 5<br />

** result : ** 25.346260387811636<br />


In [0]:
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
# **********************READ DATA**********************************************
def readData(trainFile, testFile):
    train = io.loadmat(trainFile)
    test = io.loadmat(testFile)
    ###TRAIN DATA###
    train_label = np.array(train['y'][0])
    train_vector = np.array(train['x']) / 255
    # train_vector = (train_vector -np.min(train_vector)) / (np.max(train_vector)-np.min(train_vector))
    # standardization
    # train_vector  = (train_vector - np.mean(train_vector)) / (10.0 * np.std(train_vector))

    ###TEST DATA###
    test_label = np.array(test['y'][0])
    test_vector = np.array(test['x']) / 255
    return train_label, train_vector, test_label, test_vector
#**************ONE HOT*********************************************************
def oneHot(label, class_size):
    y = label.shape[0]
    label_matrix = np.zeros([y, class_size], dtype=int)
    for i in range(y):
        correct_class = label[i]
        label_matrix[i][correct_class] = 1
    return label_matrix
#**********************SIGMOID FUNCTION****************************************
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivative_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))
# ************************ReLu FUNCTION****************************************
def relu(x):
    return np.maximum(x, 0)

def derivative_relu(x):
    x[x <= 0] = 0
    x[x > 0] = 1
    return x
#*************************TANH FUNCTION****************************************
def tanh(X):
    return np.tanh(X)
    
def derivative_tanh(x):
    return 1.0 - np.tanh(x) ** 2
# ***************SOFTMAX FUNCTION**********************************************
def softmax(out_array):
    for i in range(out_array.shape[0]):
        out_array[i] = np.exp(out_array[i]) / np.sum(np.exp(out_array[i]), axis=0)
    return out_array

def derivative_softmax(preds):
    return preds * (1 - preds)
#*****************LOSS FUNCTION************************************************
def loss(train_label, a1, size):
    result = -np.sum(np.multiply(train_label, np.log(a1)) + np.multiply(1 - train_label, np.log(1 - a1))) / size
    return result
# ******************************************************************************
# update parameters
def update_parameters_NN(parameters, grads, learning_rate=0.01):
    parameters = {"weight1": parameters["weight1"] - learning_rate * grads["dweight1"],
                  "bias1": parameters["bias1"] - learning_rate * grads["dbias1"],
                  "weight2": parameters["weight2"] - learning_rate * grads["dweight2"],
                  "bias2": parameters["bias2"] - learning_rate * grads["dbias2"]}
    return parameters
#################################################################################
# intialize parameters and layer sizes
def initialize_parameters_and_layer_sizes_NN(input_s, output_s, hidden_size):
    parameters = {"weight1": np.random.randn(input_s, hidden_size),
                  "bias1": 0.0,
                  "weight2": np.random.randn(hidden_size, output_s),
                  "bias2": 0.0}
    return parameters
#################################################################################
def forward_propagation_NN( train_vector, parameters):
    # this formula : oi = wijxj + bi
    Z1 = train_vector.dot(parameters["weight1"]) + parameters["bias1"]
    A1 = softmax(Z1/100)
    Z2 = A1.dot(parameters["weight2"]) + parameters["bias2"]
    A2 = softmax(Z2/100)

    cache = {"Z1": Z1,#h1
             "A1": A1,#a1
             "Z2": Z2,
             "A2": A2}

    return A2, cache
#################################################################################
# Compute cost
def compute_cost_NN(A2, Y, size):
    logprobs = np.multiply(np.log(A2),Y)
    cost = -np.sum(logprobs)/size
    return cost
#################################################################################
# Backward Propagation
def backward_propagation_NN(parameters, cache, X, Y,size):
    da2 = -((np.divide(Y,cache["A2"]) - np.divide(1-Y,1-cache["A2"])))/100
    dh2 = da2 * derivative_softmax(cache["A2"])
    dw2 = cache["A1"].T.dot(dh2)
    db2 = np.sum(dh2)
    dh1 = derivative_softmax(cache["Z1"]) * db2
    dw1 = np.matmul(X.T, dh1)
    db1 = np.sum(dh1)
    grads = {"dweight1": dw1,
             "dbias1": db1,
             "dweight2": dw2,
             "dbias2": db2}
    return grads
##################################################################################
def accuracy(x_test,y_test,parameters):
    preds = softmax((parameters["weight1"]) + parameters["bias1"] / 100)
    preds2 = softmax(preds.dot(parameters["weight2"]) + parameters["bias2"] / 100)
    count = 0
    for i, j in zip(y_test, np.argmax(preds2, axis=1)):
        if i == j:
            count += 1
    print("result accuracy")
    print(count / y_test.shape[0] * 100)
##################################################################################
def m_nn(x_train, y_train, x_test, y_test, epoch, learning_rate,batch_size,output_size, input_size , hidden_size):

    # initialize parameters and layer sizes
    parameters = initialize_parameters_and_layer_sizes_NN(input_size,output_size,hidden_size)
    for i in range(epoch):
        # forward propagation
        size = x_train.shape[0]
        A2, cache = forward_propagation_NN(x_train, parameters)
        # compute cost
        compute_cost_NN(A2, y_train,size)
        # backward propagation
        grads = backward_propagation_NN(parameters,cache, x_train, y_train,size)
        # update parameters
        parameters = update_parameters_NN(parameters, grads)
        '''
        if i % 100 == 0:
            preds = softmax((parameters["weight1"]) + parameters["bias1"] / 100)
            preds2 = softmax(preds.dot(parameters["weight2"]) + parameters["bias2"] /100)
            count = 0
            for i, j in zip(y_test, np.argmax(preds2, axis=1)):
                if i == j:
                    count += 1

            print(count / y_test.shape[0] * 100)
        '''
    accuracy(x_test, y_test,parameters)

def main():
    # ********ASSIGN VALUE*******************************
    train_label, train_vector, test_label, test_vector = readData('train.mat', 'test.mat')
    epoch_size = 1000
    train_label = oneHot(train_label, 5)
    learning_rate = 0.01
    batch_size = 126
    output_size = 5
    input_size = 768
    hidden_size = 50
    # other typhical value 0.001, 0.01, 0.03, 0.1, 0.3, 1 etc.
    m_nn(train_vector, train_label, test_vector, test_label, epoch_size,learning_rate, batch_size, output_size, input_size , hidden_size)

if __name__ == '__main__':
    main()